In [0]:
%sql
drop table if exists first_project.gold.total_orders_by_date;
create table first_project.gold.total_orders_by_date
with first_buy as (
  select a.customerkey,
        min(orderdate) as first_buy_date
  from first_project.silver.orders a
  group by a.customerkey
),

total_orders_by_date as (
  select a.orderdate,
        count(*) as total_order_by_date,
        count(distinct a.customerkey) as total_distinct_customer_by_date,
        count(distinct b.ProductKey) as total_distinct_product_by_date,
        count(c.customerkey) as total_new_customers_by_date,
        round(sum(b.total_price_usd), 2) as total_revenue_usd_by_date,
        sum(b.quantity) as total_quantity_by_date
  from first_project.silver.orders a
  join first_project.silver.orderrows b on a.orderkey = b.OrderKey
  left join first_buy c on a.customerkey = c.customerkey and a.orderdate = c.first_buy_date
  where a.orderdate > add_months(trunc(current_date(),'year'),-24)
  group by a.orderdate
)

select a.*
from total_orders_by_date a